In [1]:
from sympy.logic.boolalg import And, Or, Not, Implies, Equivalent
from sympy import symbols, bool_map
from sympy.logic.inference import satisfiable
from itertools import combinations
from belief_agent import BeliefBase
from math import factorial

**KB: Robert does well in the exam if and only if he is prepared or lucky.**
Robert does not do well the exam.

**Question: KB |= φ?**  **(KB entails phi)**

1. (-r $\vee$ p $\vee$ s) $\wedge$ (p $\wedge$ r) $\wedge$ (s $\wedge$ r) $\wedge$ $\neg$ r, formalisation of KB as CNF.

2.(-r $\vee$ p $\vee$ s) $\wedge$ (p $\wedge$ r) $\wedge$ (s $\wedge$ r) $\wedge$ r $\wedge$ p, CNF for contradiction.

3. clauses:= (-r $\vee$ p $\vee$ s), (p $\wedge$ r), (s $\wedge$ r), r, p.

4. clauses:=clauses U {r} from resolving (¬p $\vee$ r) with p.

5. clauses:=clauses U {1} from resolving r with $\neg$ r.

6. Result: empty clause, so KB |= φ.

In [2]:
belief_base = BeliefBase('q p')
symb = belief_base.symbols
q, p = symb

belief_base.expand(p, 1)
belief_base.expand(q, 2)
belief_base.expand(Implies(p, q), 3)
print("Belief Base:", belief_base)

Belief Base: <belief_agent.BeliefBase object at 0x11533ccd0>


In [3]:
belief_base.contract(q)


Sorted beliefs: [(q | ~p, 3), (q, 2), (p, 1)]
q | ~p False
q True
q
p False


In [148]:
satisf = satisfiable(belief_base.to_CNF())
key = tuple([satisf[belief_base.symbols[i]] for i in range(len(belief_base.symbols))])

In [149]:
belief_base.priorities(key)

TypeError: 'dict' object is not callable

In [53]:
from sympy.logic.boolalg import And, Or, Not, Implies, Equivalent, to_cnf
from sympy import symbols
from sympy.logic.inference import satisfiable
from utils import generate_combinations

class BeliefBase:
    def __init__(self, atomic_symbols: str):
        self.beliefs = set()
        self.beliefs_priorities = set()
        self.symbols = symbols(atomic_symbols)
        self.priorities = {}

    def expand(self, belief, priority):
        cnf_belief = to_cnf(belief)
        self.beliefs.add((cnf_belief))
        self.beliefs_priorities.add((cnf_belief, priority))
        self.priorities[belief] = priority

    def remove(self, belief, belief_set=None):
        cnf_belief = to_cnf(belief)
        if belief_set is None:
            if cnf_belief in self.beliefs:
                self.beliefs.remove(cnf_belief)
        else:
            if cnf_belief in belief_set:
                belief_set.remove(cnf_belief)

    def contract(self, extract_belief):

        if extract_belief in self.beliefs:
            # We remove the belief both from our priority list and our belief set
            self.remove(extract_belief)
            self.beliefs_priorities.remove((extract_belief, self.priorities[extract_belief]))

            sorted_beliefs = sorted(self.beliefs_priorities, key=lambda x: (-x[1]))
            while self.check_entailment(extract_belief):
                for (belief, pri) in sorted_beliefs:
                    belief_not_entailed = satisfiable(And(Not(extract_belief), belief))
                    print("\nis_belief_entailed == True??", belief_not_entailed == True)
                    print("belief:",belief, "  is belief entailed?:", belief_not_entailed)
                    if belief_not_entailed != False:
                        print("Belief to be removed:", belief)
                        self.remove(belief)
                        sorted_beliefs.remove((to_cnf(belief), pri))
                        break


    def revise(self, belief):
        # Levi's identity
        self.contract(Not(belief))
        self.expand(belief)

    def check_entailment(self, formula, belief_base=None):
        if belief_base is None:
            cnf_formula = to_cnf(formula)
            belief_base_cnf = to_cnf(self.beliefs)
            new_set = And(*belief_base_cnf, Not(cnf_formula))
            return not satisfiable(new_set)
        else:
            cnf_formula = to_cnf(formula)
            belief_base_cnf = to_cnf(belief_base)
            new_set = And(belief_base_cnf, Not(cnf_formula))
            return not satisfiable(new_set)         

# Test
if __name__ == "__main__":
    belief_base = BeliefBase('p q')  
    p, q = belief_base.symbols
    # Expand belief base
    belief_base.expand(p, 1)
    belief_base.expand(q, 2)
    belief_base.expand(Implies(p, q), 3)
    print(belief_base.beliefs)
    
    print("\n Contraction:")
    belief_base.contract(q)
    print("\nNew beliefbalse:")
    print(belief_base.beliefs) 

    """ print("Belief Base:", belief_base)
    print("Belief Base CNF:", belief_base.to_CNF())

    # Check entailment of a formula
    formula = Implies(A, C)
    print("Entailment of {}: {}".format(formula, belief_base.check_entailment(formula))) """

{q | ~p, q, p}

 Contraction:

is_belief_entailed == True?? False
belief: q | ~p   is belief entailed?: {p: False, q: False}
Belief to be removed: q | ~p

New beliefbalse:
{p}


In [30]:
print(belief_base.beliefs_priorities)
print(belief_base.priorities)

{(q, 2), (q | ~p, 3), (p, 1)}
{p: 1, q: 2, Implies(p, q): 3}


In [13]:
lista = sorted(belief_base.beliefs_priorities, key=lambda x: (-x[1]))

In [14]:
lista

[(B | C, 5), (C, 5), (C, 3), (~B, 2), (B, 2), (A, 1)]

In [16]:
lista.remove((B | C, lista[0][1]))

In [17]:
lista

[(C, 5), (C, 3), (~B, 2), (B, 2), (A, 1)]